In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
from imblearn.over_sampling import SMOTE

In [2]:
#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data2/Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

#스케일링 먼저 하고 ndarray를 dataframe으로 변환
scaler = StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

In [3]:
Count= int(322/10)*83
for i in range(10):
    #마지막 그룹은 34명
    if i==9:
        X_test=X.iloc[Count*i:-1]
        X_train=X.drop(X.index[Count*i:-1])
        y_test=y.iloc[Count*i:-1]
        y_train=y.drop(y.index[Count*i:-1])
    
    #모든 그룹은 32명씩 (마지막은 34)
    X_test=X.iloc[Count*i:Count*(i+1)]
    X_train=X.drop(X.index[Count*i:Count*(i+1)])
    y_test=y.iloc[Count*i:Count*(i+1)]
    y_train=y.drop(y.index[Count*i:Count*(i+1)])
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train,y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    
    #모델 정의, 예측
    #n_neighbors는 여러 개 해보고 제일 잘 나오는걸로 구하래요~ 월욜에합시다
    model = KNeighborsClassifier(n_neighbors=7)
    model.fit(X_train, y_train)
    
    predict=model.predict(X_test)
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))
    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print()
    
    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=f1
    Result[i][2]=list[0]
    Result[i][3]=list[1]

[0]Accuracy : 0.3677782352133937
[0]F1score : 0.3560665570841963
[0]Precision : 0.3557882394876165
[0]Recall : 0.3677782352133937

[1]Accuracy : 0.37166666666666665
[1]F1score : 0.36389178391080007
[1]Precision : 0.37403055109470285
[1]Recall : 0.37166666666666665

[2]Accuracy : 0.4335459861775651
[2]F1score : 0.4149081810905634
[2]Precision : 0.42428241496947616
[2]Recall : 0.4335459861775651

[3]Accuracy : 0.33426443202979517
[3]F1score : 0.3134723911885604
[3]Precision : 0.3203236850666322
[3]Recall : 0.33426443202979517

[4]Accuracy : 0.43169158604514807
[4]F1score : 0.4303870366912557
[4]Precision : 0.4323203759360073
[4]Recall : 0.43169158604514807

[5]Accuracy : 0.35661117877926646
[5]F1score : 0.34731609672172153
[5]Precision : 0.34871520370093884
[5]Recall : 0.35661117877926646

[6]Accuracy : 0.3681430312880943
[6]F1score : 0.36359757398810594
[6]Precision : 0.3641754913605651
[6]Recall : 0.3681430312880942

[7]Accuracy : 0.3832982456140351
[7]F1score : 0.37441469722446785
[7]

In [4]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.367778,0.356067,0.355788,0.367778
1,0.371667,0.363892,0.374031,0.371667
2,0.433546,0.414908,0.424282,0.433546
3,0.334264,0.313472,0.320324,0.334264
4,0.431692,0.430387,0.432320,0.431692
5,0.356611,0.347316,0.348715,0.356611
6,0.368143,0.363598,0.364175,0.368143
7,0.383298,0.374415,0.375869,0.383298
8,0.442064,0.443133,0.446281,0.442064
9,0.371027,0.356716,0.358845,0.371027


In [5]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.38600905508948796
Average of F1-Score 0.37639028805589475
Average of Precision 0.38006305246846966
Average of Recall 0.38600905508948796
